In [7]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import mmcv
import importlib
import torch
import torchvision.transforms as transforms
from mmcv import Config
from mmcls.datasets.pipelines import Compose
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
from PIL import Image
import heapq
from mmseg.apis import inference_segmentor, init_segmentor
from mmseg.core.evaluation import get_palette
from pytorch_grad_cam.utils.image import show_cam_on_image
import cv2

import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    

from scripts import generate_statistics_single
from scripts import visualization_seg_masks
from scripts import generate_statistics
from scripts import generate_cams
from scripts import utils
from scripts.utils.pipeline import get_pipeline_torchvision
from scripts import generate_segs
from scripts import evaluate_effectiveness
from scripts.generate_blurred import generate_blurred

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
configSeg = '../segmentation/ocrnet_hr48_512x512_carparts_noflip/ocrnet_hr48_512x512_carparts_noflip.py'
checkpointSeg = '../segmentation/ocrnet_hr48_512x512_carparts_noflip/latest.pth'
configCAMResOG = '../CAMModels/resnet/compCars_Original/resnet50_b128x2_compcars-original-split.py'
checkpointCAMResOG = '../CAMModels/resnet/compCars_Original/latest.pth'
configCAMResCol = '../CAMModels/resnet/compCars_Color/resnet50_b128x2_compcars-color-split.py'
checkpointCAMResCol = '../CAMModels/resnet/compCars_Color/latest.pth'
configCAMResWeb = '../CAMModels/resnet/compCars_Web/resnet50_b128x2_compcars-web-original-split.py'
checkpointCAMResWeb = '../CAMModels/resnet/compCars_Web/latest.pth'
configCAMResStanford = '../CAMModels/resnet/stanfordCars_Original/resnet50_b128x2_stanford.py'
checkpointCAMResStanford = '../CAMModels/resnet/stanfordCars_Original/latest.pth'

configCAMSwinSmallOG = '../CAMModels/swinSmall/compCars_Original/swin-small_b128x2_compcars-original-split.py'
checkpointCAMSwinSmallOG = '../CAMModels/swinSmall/compCars_Original/latest.pth'
configCAMSwinSmallCol = '../CAMModels/swinSmall/compCars_Color/swin-small_b128x2_compcars-color-split.py'
checkpointCAMSwinSmallCol = '../CAMModels/swinSmall/compCars_Color/latest.pth'
configCAMSwinSmallWeb = '../CAMModels/swinSmall/compCars_WebOriginal/swin-small_b128x2_compcars-web-original-split.py'
checkpointCAMSwinSmallWeb = '../CAMModels/swinSmall/compCars_WebOriginal/latest.pth'
configCAMSwinSmallStanford = '../CAMModels/swinSmall/stanfordCars_Original/swin-small_b128x2_stanford.py'
checkpointCAMSwinSmallStanford = '../CAMModels/swinSmall/stanfordCars_Original/latest.pth'

configCAMSwinBaseOG = '../CAMModels/swinBase/compCars_Original/swin-base_b128x2_compcars-original-split.py'
checkpointCAMSwinBaseOG = '../CAMModels/swinBase/compCars_Original/latest.pth'
configCAMSwinBaseCol = '../CAMModels/swinBase/compCars_Color/swin-base_b128x2_compcars-color-split.py'
checkpointCAMSwinBaseCol = '../CAMModels/swinBase/compCars_Color/latest.pth'
configCAMSwinBaseWeb = '../CAMModels/swinBase/compCars_WebOriginal/swin-base_b128x2_compcars-web-original-split.py'
checkpointCAMSwinBaseWeb = '../CAMModels/swinBase/compCars_WebOriginal/latest.pth'
configCAMSwinBaseStanford = '../CAMModels/swinBase/stanfordCars_Original/swin-base_b128x2_stanford.py'
checkpointCAMSwinBaseStanford = '../CAMModels/swinBase/stanfordCars_Original/latest.pth'

annfileTrain = '../annfileTrain.txt'
annfileVal = '../annfileVal.txt'
annfileValGt = '../annfileValGt.txt'
imgNameTrain='Acura_Acura_ILX_2a6cce617fc27d.jpg'
imgNameVal = 'Acura_Acura_ILX_3c11ef9e42931b.jpg'
imgPathTrain=os.path.join(imgRootTrain, imgNameTrain)
imgPathVal=os.path.join(imgRootVal, imgNameVal)

annfileValOriginal = '../data/CompCars_sv_original_split/meta/val.txt'
annfileValColor = '../data/CompCars_sv_color_split/meta/val.txt'
annfileValWeb = '../data/CompCars_web_original_split/meta/val.txt'
annfileValStanford = '../data/StanfordCars_original_split/meta/val.txt'

paletteName='Comp_Original_Ocrnet_Carparts_Noflip'

segDataPathNoScaleOG = 'G:/XAI-Vehicle-Classification/preGenData/Segmentations/CompCars_Original/valAllSegsNoScale.npz'
segDataPathNoScaleWeb = 'G:/XAI-Vehicle-Classification/preGenData/Segmentations/CompCars_Web/valAllSegsNoScale.npz'
segDataPathNoScaleStanfordOG = 'G:/XAI-Vehicle-Classification/preGenData/Segmentations/StanfordCars_Original/valAllSegsNoScale.npz'

imgRootValOriginal = '../data/CompCars_sv_original_split/val'
imgRootValColor = '../data/CompCars_sv_color_split/val'
imgRootValWeb = '../data/CompCars_web_original_split/val'
imgRootValStandford = '../data/StanfordCars_original_split/val'

blurWhiteBackgroundImgRootOriginal = '../data/CompCars_sv_original_split/blurred/blurredBackgroundWhite'
blurWhiteBackgrdHoodFBumperImgRootOriginal = '../data/CompCars_sv_original_split/blurred/blurredBackground_Hood_FBumperWhite'
blurWhiteFLLightHoodFBumperImgRootOriginal = '../data/CompCars_sv_original_split/blurred/blurredFLLight_Hood_FBumperWhite'
blurWhiteFLLightImgRootOriginal = '../data/CompCars_sv_original_split/blurred/blurredFrontLeftLightWhite'

blurWhiteBackgroundImgRootWeb = '../data/CompCars_web_original_split/blurred/blurredBackgroundWhite'
blurWhiteBackgrdHoodFBumperImgRootWeb = '../data/CompCars_web_original_split/blurred/blurredBackground_Hood_FBumperWhite'
blurWhiteFLLightHoodFBumperImgRootWeb = '../data/CompCars_web_original_split/blurred/blurredFLLight_Hood_FBumperWhite'
blurWhiteFLLightImgRootWeb = '../data/CompCars_web_original_split/blurred/blurredFrontLeftLightWhite'

blurWhiteBackgroundImgRootStanford = '../data/StanfordCars_original_split/blurred/blurredBackgroundWhite'
blurWhiteBackgrdHoodFBumperImgRootStanford = '../data/StanfordCars_original_split/blurred/blurredBackground_Hood_FBumperWhite'
blurWhiteFLLightHoodFBumperImgRootStanford = '../data/StanfordCars_original_split/blurred/blurredFLLight_Hood_FBumperWhite'
blurWhiteFLLightImgRootStanford = '../data/StanfordCars_original_split/blurred/blurredFrontLeftLightWhite'

In [4]:
igR = imgRootValOriginal
cfgC = configCAMResOG
ckpC = checkpointCAMResOG
svPre = '../preGenData/CompCars_Original/ResNet/GPU/'
#generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++GPU', '--method', 'gradcam++', '--device', 'cuda'])
#generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsXGradGPU', '--method', 'xgradcam', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGPU', '--method', 'eigencam', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGradGPU', '--method', 'eigengradcam', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsLayerGPU', '--method', 'layercam', '--device', 'cuda'])

Method for CAM generation: eigencam, eigen-smooth:False, aug-smooth:False, vit-like:False
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 3.3 task/s, elapsed: 4092s, ETA:     0sSave Split file for Cams
Save generated CAMs to ../preGenData/CompCars_Original/ResNet/GPU\valAllCAMsEigenGPU.npz
Method for CAM generation: eigengradcam, eigen-smooth:False, aug-smooth:False, vit-like:False
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 3.3 task/s, elapsed: 3992s, ETA:     0sSave Split file for Cams
Save generated CAMs to ../preGenData/CompCars_Original/ResNet/GPU\valAllCAMsEigenGradGPU.npz
Method for CAM generation: layercam, eigen-smooth:False, aug-smooth:False, vit-like:False
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 22.1 task/s

In [5]:
igR = imgRootValOriginal
cfgC = configCAMSwinSmallOG
ckpC = checkpointCAMSwinSmallOG
svPre = '../preGenData/CompCars_Original/SwinSmall/GPU/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++GPU', '--method', 'gradcam++', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsXGradGPU', '--method', 'xgradcam', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGPU', '--method', 'eigencam', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGradGPU', '--method', 'eigengradcam', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsLayerGPU', '--method', 'layercam', '--vit-like', '--device', 'cuda'])

Method for CAM generation: gradcam++, eigen-smooth:False, aug-smooth:False, vit-like:True
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 12.7 task/s, elapsed: 1051s, ETA:     0sSave Split file for Cams
Save generated CAMs to ../preGenData/CompCars_Original/SwinSmall/GPU\valAllCAMsGrad++GPU.npz
Method for CAM generation: xgradcam, eigen-smooth:False, aug-smooth:False, vit-like:True
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 12.7 task/s, elapsed: 1047s, ETA:     0sSave Split file for Cams
Save generated CAMs to ../preGenData/CompCars_Original/SwinSmall/GPU\valAllCAMsXGradGPU.npz
Method for CAM generation: eigencam, eigen-smooth:False, aug-smooth:False, vit-like:True
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 12.6 task/s, el

In [6]:
igR = imgRootValOriginal
cfgC = configCAMSwinBaseOG
ckpC = checkpointCAMSwinBaseOG
svPre = '../preGenData/CompCars_Original/SwinBase/GPU/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++GPU', '--method', 'gradcam++', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsXGradGPU', '--method', 'xgradcam', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGPU', '--method', 'eigencam', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGradGPU', '--method', 'eigengradcam', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsLayerGPU', '--method', 'layercam', '--vit-like', '--device', 'cuda'])

Method for CAM generation: gradcam++, eigen-smooth:False, aug-smooth:False, vit-like:True
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 12.7 task/s, elapsed: 1047s, ETA:     0sSave Split file for Cams
Save generated CAMs to ../preGenData/CompCars_Original/SwinBase/GPU\valAllCAMsGrad++GPU.npz
Method for CAM generation: xgradcam, eigen-smooth:False, aug-smooth:False, vit-like:True
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 12.6 task/s, elapsed: 1057s, ETA:     0sSave Split file for Cams
Save generated CAMs to ../preGenData/CompCars_Original/SwinBase/GPU\valAllCAMsXGradGPU.npz
Method for CAM generation: eigencam, eigen-smooth:False, aug-smooth:False, vit-like:True
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 8.5 task/s, elap

In [12]:
generate_segs.main([imgRootValOriginal, configSeg, checkpointSeg,
                    '-s', 'G:/XAI-Vehicle-Classification/preGenData/Segmentations/CompCars_Color/valAllSegsNoScale'])
generate_segs.main([imgRootValColor, configSeg, checkpointSeg,
                    '-s', 'G:/XAI-Vehicle-Classification/preGenData/Segmentations/CompCars_Color/valAllSegsNoScale'])
generate_segs.main([imgRootValWeb, configSeg, checkpointSeg,
                    '-s', 'G:/XAI-Vehicle-Classification/preGenData/Segmentations/CompCars_Web/valAllSegsNoScale'])

Saving resulting segmentation masks in G:/XAI-Vehicle-Classification/preGenData/Segmentations/CompCars_Original


C:\Users\Linus\Desktop\XAI-Vehicle-Classification\scripts\utils\io.py:106: UserWarning: Using generate_split_files will redirect the call to generate_ann_file. Consider directly calling that function
  warnings.warn("Using generate_split_files will redirect the call to generate_ann_file. Consider directly calling that function")


Creating annotation file at G:/XAI-Vehicle-Classification/preGenData/Segmentations/CompCars_Original\valAllSegsNoScale.txt
Calculating segmentation results.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 8.7 task/s, elapsed: 1532s, ETA:     0s
Saving results at G:/XAI-Vehicle-Classification/preGenData/Segmentations/CompCars_Original\valAllSegsNoScale.npz
Saving resulting segmentation masks in G:/XAI-Vehicle-Classification/preGenData/Segmentations/CompCars_Color
Creating annotation file at G:/XAI-Vehicle-Classification/preGenData/Segmentations/CompCars_Color\valAllSegsNoScale.txt
Calculating segmentation results.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13379/13379, 8.8 task/s, elapsed: 1528s, ETA:     0s
Saving results at G:/XAI-Vehicle-Classification/preGenData/Segmentations/CompCars_Color\valAllSegsNoScale.npz
Saving resulting segmentation masks in G:/XAI-Vehicle-Classification/preGenData/Segmentations/CompCars_Web
Creating annotation file at G:/XAI-Vehicle-Classification/preGenData/Segmentations/CompCar

In [11]:
evaluate_effectiveness.evaluate_blurred(imgRootValOriginal, configCAMResOG, checkpointCAMResOG, annfileValOriginal,
                                       segDataPathNoScaleOG, 'background', saveImgs=True, saveDir='blurredBackgroundOG',
                                       segConfig=configSeg, segCheckpoint=checkpointSeg)

Evaluating original model vs blurred where segments background are blurred.
Created filtered annotation file at blurredBackgroundOG\annfile_filtered.txt
Computing blurred images on demand.
Setting up configs for Blurred Dataset
Blurred Images will be saved to blurredBackgroundOG\blurredImgs
Model already on GPU
Computing Evaluation Metrics for Original Dataset
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 184.3 task/s, elapsed: 72s, ETA:     0s
Saving json data to blurredBackgroundOG\eval_results_original.json
Computing Evaluation Metrics for Blurred Dataset
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 24.4 task/s, elapsed: 545s, ETA:     0s
Saving json data to blurredBackgroundOG\eval_results_blurred.json
Add total Change and improvement of original over blurred
Saving evaluation results to blurredBackgroundOG\evalBlurred.xlsx
Removing temporary filtered annotation file blurredBackgroundOG\annfile_filtered.txt


In [4]:
igR = 'C:/Users/Linus/Desktop/XAI-Vehicle-Classification/data/CompCars_sv_original_split/blurred/blurredBackground'

cfgC = configCAMResOG
ckpC = checkpointCAMResOG
svPre = '../preGenData/blurredBackground/CompCars_Original/ResNet/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++GPU', '--method', 'gradcam++'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGPU', '--method', 'eigencam'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsLayerGPU', '--method', 'layercam'])

cfgC = configCAMSwinBaseOG
ckpC = checkpointCAMSwinBaseOG
svPre = '../preGenData/blurredBackground/CompCars_Original/SwinBase/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++GPU', '--method', 'gradcam++', '--vit-like'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGPU', '--method', 'eigencam', '--vit-like'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsLayerGPU', '--method', 'layercam', '--vit-like'])

cfgC = configCAMSwinSmallOG
ckpC = checkpointCAMSwinSmallOG
svPre = '../preGenData/blurredBackground/CompCars_Original/SwinSmall/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++GPU', '--method', 'gradcam++', '--vit-like'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGPU', '--method', 'eigencam', '--vit-like'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsLayerGPU', '--method', 'layercam', '--vit-like'])


Method for CAM generation: eigencam, eigen-smooth:False, aug-smooth:False, vit-like:True
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 2.5 task/s, elapsed: 5271s, ETA:     0s
Save Split file for Cams
Creating annotation file at ../preGenData/CompCars_Original/SwinBase\valAllCAMsEigenGPU.txt
Save generated CAMs to ../preGenData/CompCars_Original/SwinBase\valAllCAMsEigenGPU.npz


C:\Users\Linus\Desktop\XAI-Vehicle-Classification\scripts\utils\io.py:108: UserWarning: Using generate_split_files will redirect the call to generate_ann_file. Consider directly calling that function
  warnings.warn("Using generate_split_files will redirect the call to generate_ann_file. Consider directly calling that function")


Method for CAM generation: layercam, eigen-smooth:False, aug-smooth:False, vit-like:True
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 2.4 task/s, elapsed: 5613s, ETA:     0s
Save Split file for Cams
Creating annotation file at ../preGenData/CompCars_Original/SwinBase\valAllCAMsLayerGPU.txt
Save generated CAMs to ../preGenData/CompCars_Original/SwinBase\valAllCAMsLayerGPU.npz
Method for CAM generation: gradcam++, eigen-smooth:False, aug-smooth:False, vit-like:True
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 3.3 task/s, elapsed: 4086s, ETA:     0s
Save Split file for Cams
Creating annotation file at ../preGenData/CompCars_Original/SwinSmall\valAllCAMsGrad++GPU.txt
Save generated CAMs to ../preGenData/CompCars_Original/SwinSmall\valAllCAMsGrad++GPU.npz
Method for CAM generation: eigencam, eigen-smooth:False, au

In [3]:
igR = 'C:/Users/Linus/Desktop/XAI-Vehicle-Classification/data/CompCars_sv_original_split/blurred/blurredFrontLeftLight'

cfgC = configCAMResOG
ckpC = checkpointCAMResOG
svPre = '../preGenData/blurfllight/CompCars_Original/ResNet/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++GPU', '--method', 'gradcam++'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGPU', '--method', 'eigencam'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsLayerGPU', '--method', 'layercam'])

cfgC = configCAMSwinBaseOG
ckpC = checkpointCAMSwinBaseOG
svPre = '../preGenData/blurfllight/CompCars_Original/SwinBase/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++GPU', '--method', 'gradcam++', '--vit-like'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGPU', '--method', 'eigencam', '--vit-like'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsLayerGPU', '--method', 'layercam', '--vit-like'])

cfgC = configCAMSwinSmallOG
ckpC = checkpointCAMSwinSmallOG
svPre = '../preGenData/blurfllight/CompCars_Original/SwinSmall/'
#generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++GPU', '--method', 'gradcam++', '--vit-like'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGPU', '--method', 'eigencam', '--vit-like'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsLayerGPU', '--method', 'layercam', '--vit-like'])


Method for CAM generation: eigencam, eigen-smooth:False, aug-smooth:False, vit-like:True
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 3.7 task/s, elapsed: 3627s, ETA:     0s
Save Split file for Cams
Creating annotation file at ../preGenData/blurfllight/CompCars_Original/SwinSmall\valAllCAMsEigenGPU.txt
Save generated CAMs to ../preGenData/blurfllight/CompCars_Original/SwinSmall\valAllCAMsEigenGPU.npz


C:\Users\Linus\Desktop\XAI-Vehicle-Classification\scripts\utils\io.py:108: UserWarning: Using generate_split_files will redirect the call to generate_ann_file. Consider directly calling that function
  warnings.warn("Using generate_split_files will redirect the call to generate_ann_file. Consider directly calling that function")


Method for CAM generation: layercam, eigen-smooth:False, aug-smooth:False, vit-like:True
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 3.3 task/s, elapsed: 4085s, ETA:     0s
Save Split file for Cams
Creating annotation file at ../preGenData/blurfllight/CompCars_Original/SwinSmall\valAllCAMsLayerGPU.txt
Save generated CAMs to ../preGenData/blurfllight/CompCars_Original/SwinSmall\valAllCAMsLayerGPU.npz


In [4]:
igR = 'C:/Users/Linus/Desktop/XAI-Vehicle-Classification/data/CompCars_sv_original_split/blurred/blurredFLLight_Hood_FBumper'

cfgC = configCAMResOG
ckpC = checkpointCAMResOG
svPre = '../preGenData/blurredFLLight_Hood_FBumper/CompCars_Original/ResNet/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++GPU', '--method', 'gradcam++'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGPU', '--method', 'eigencam'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsLayerGPU', '--method', 'layercam'])

cfgC = configCAMSwinBaseOG
ckpC = checkpointCAMSwinBaseOG
svPre = '../preGenData/blurredFLLight_Hood_FBumper/CompCars_Original/SwinBase/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++GPU', '--method', 'gradcam++', '--vit-like'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGPU', '--method', 'eigencam', '--vit-like'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsLayerGPU', '--method', 'layercam', '--vit-like'])

cfgC = configCAMSwinSmallOG
ckpC = checkpointCAMSwinSmallOG
svPre = '../preGenData/blurredFLLight_Hood_FBumper/CompCars_Original/SwinSmall/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++GPU', '--method', 'gradcam++', '--vit-like'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGPU', '--method', 'eigencam', '--vit-like'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsLayerGPU', '--method', 'layercam', '--vit-like'])

Method for CAM generation: gradcam++, eigen-smooth:False, aug-smooth:False, vit-like:False
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 4.4 task/s, elapsed: 3041s, ETA:     0s
Save Split file for Cams
Creating annotation file at ../preGenData/blurredFLLight_Hood_FBumper/CompCars_Original/ResNet\valAllCAMsGrad++GPU.txt
Save generated CAMs to ../preGenData/blurredFLLight_Hood_FBumper/CompCars_Original/ResNet\valAllCAMsGrad++GPU.npz
Method for CAM generation: eigencam, eigen-smooth:False, aug-smooth:False, vit-like:False
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 3.0 task/s, elapsed: 4403s, ETA:     0s
Save Split file for Cams
Creating annotation file at ../preGenData/blurredFLLight_Hood_FBumper/CompCars_Original/ResNet\valAllCAMsEigenGPU.txt
Save generated CAMs to ../preGenData/blurredFLLight_Hood_FBumper/Com

In [3]:
igR = 'C:/Users/Linus/Desktop/XAI-Vehicle-Classification/data/CompCars_sv_original_split/blurred/blurredBackground_Hood_FBumper'

cfgC = configCAMResOG
ckpC = checkpointCAMResOG
svPre = '../preGenData/blurredBackground_Hood_FBumper/CompCars_Original/ResNet/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++GPU', '--method', 'gradcam++'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGPU', '--method', 'eigencam'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsLayerGPU', '--method', 'layercam'])

cfgC = configCAMSwinBaseOG
ckpC = checkpointCAMSwinBaseOG
svPre = '../preGenData/blurredBackground_Hood_FBumper/CompCars_Original/SwinBase/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++GPU', '--method', 'gradcam++', '--vit-like'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGPU', '--method', 'eigencam', '--vit-like'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsLayerGPU', '--method', 'layercam', '--vit-like'])

cfgC = configCAMSwinSmallOG
ckpC = checkpointCAMSwinSmallOG
svPre = '../preGenData/blurredBackground_Hood_FBumper/CompCars_Original/SwinSmall/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++GPU', '--method', 'gradcam++', '--vit-like'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGPU', '--method', 'eigencam', '--vit-like'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsLayerGPU', '--method', 'layercam', '--vit-like'])

Method for CAM generation: gradcam++, eigen-smooth:False, aug-smooth:False, vit-like:True
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 3.2 task/s, elapsed: 4113s, ETA:     0s
Save Split file for Cams
Creating annotation file at ../preGenData/blurredBackground_Hood_FBumper/CompCars_Original/SwinSmall\valAllCAMsGrad++GPU.txt
Save generated CAMs to ../preGenData/blurredBackground_Hood_FBumper/CompCars_Original/SwinSmall\valAllCAMsGrad++GPU.npz


C:\Users\Linus\Desktop\XAI-Vehicle-Classification\scripts\utils\io.py:108: UserWarning: Using generate_split_files will redirect the call to generate_ann_file. Consider directly calling that function
  warnings.warn("Using generate_split_files will redirect the call to generate_ann_file. Consider directly calling that function")


Method for CAM generation: eigencam, eigen-smooth:False, aug-smooth:False, vit-like:True
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 3.8 task/s, elapsed: 3484s, ETA:     0s
Save Split file for Cams
Creating annotation file at ../preGenData/blurredBackground_Hood_FBumper/CompCars_Original/SwinSmall\valAllCAMsEigenGPU.txt
Save generated CAMs to ../preGenData/blurredBackground_Hood_FBumper/CompCars_Original/SwinSmall\valAllCAMsEigenGPU.npz
Method for CAM generation: layercam, eigen-smooth:False, aug-smooth:False, vit-like:True
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 3.4 task/s, elapsed: 3889s, ETA:     0s
Save Split file for Cams
Creating annotation file at ../preGenData/blurredBackground_Hood_FBumper/CompCars_Original/SwinSmall\valAllCAMsLayerGPU.txt
Save generated CAMs to ../preGenData/blurredBackground_

In [7]:
generate_segs.main([imgRootValStandford, configSeg, checkpointSeg,
                    '-s', 'G:/XAI-Vehicle-Classification/preGenData/Segmentations/StanfordCars_Original/valAllSegsNoScale'])
# With pipeline
generate_segs.main([imgRootValStandford, configSeg, checkpointSeg, '-p', 'post', configCAMResOG,
                    '-s', 'G:/XAI-Vehicle-Classification/preGenData/Segmentations/StanfordCars_Original/valAllSegs'])

Saving resulting segmentation masks in G:/XAI-Vehicle-Classification/preGenData/Segmentations/StanfordCars_Original
Creating annotation file at G:/XAI-Vehicle-Classification/preGenData/Segmentations/StanfordCars_Original\valAllSegsNoScale.txt
Calculating segmentation results.
[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 8041/8041, 8.8 task/s, elapsed: 914s, ETA:     0s
Saving results at G:/XAI-Vehicle-Classification/preGenData/Segmentations/StanfordCars_Original\valAllSegsNoScale.npz
Saving resulting segmentation masks in G:/XAI-Vehicle-Classification/preGenData/Segmentations/StanfordCars_Original
Loading Pipeline from Config ../CAMModels/resnet/compCars_Original/resnet50_b128x2_compcars-original-split.py
Creating annotation file at G:/XAI-Vehicle-Classification/preGenData/Segmentations/StanfordCars_Original\valAllSegs.txt
Calculating segmentation results.
[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 8041/8041, 8.8 task/s, elapsed: 913s, ETA:     0s
Saving results at G:/XAI-Vehicle-Classification/preGenData/Segment

In [5]:
igR = imgRootValStandford
cfgC = configCAMResStanford
ckpC = checkpointCAMResStanford
svPre = '../preGenData/StanfordCars_Original/resnet/GPU/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMs', '--method', 'gradcam', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++', '--method', 'gradcam++', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsXGrad', '--method', 'xgradcam', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigen', '--method', 'eigencam', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGrad', '--method', 'eigengradcam', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsLayer', '--method', 'layercam', '--device', 'cuda'])


cfgC = configCAMSwinBaseStanford
ckpC = checkpointCAMSwinBaseStanford
svPre = '../preGenData/StanfordCars_Original/swinBase/GPU/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMs', '--method', 'gradcam', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++', '--method', 'gradcam++', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsXGrad', '--method', 'xgradcam', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigen', '--method', 'eigencam', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGrad', '--method', 'eigengradcam', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsLayer', '--method', 'layercam', '--vit-like', '--device', 'cuda'])


cfgC = configCAMSwinSmallStanford
ckpC = checkpointCAMSwinSmallStanford
svPre = '../preGenData/StanfordCars_Original/swinSmall/GPU/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMs', '--method', 'gradcam', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++', '--method', 'gradcam++', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsXGrad', '--method', 'xgradcam', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigen', '--method', 'eigencam', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigenGrad', '--method', 'eigengradcam', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsLayer', '--method', 'layercam', '--vit-like', '--device', 'cuda'])

Method for CAM generation: gradcam, eigen-smooth:False, aug-smooth:False, vit-like:True
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 8041/8041, 13.9 task/s, elapsed: 577s, ETA:     0s
Save Split file for Cams
Creating annotation file at ../preGenData/StanfordCars_Original/swinBase/GPU\valAllCAMsGradGPU.txt
Save generated CAMs to ../preGenData/StanfordCars_Original/swinBase/GPU\valAllCAMsGradGPU.npz


C:\Users\Linus\Desktop\XAI-Vehicle-Classification\scripts\utils\io.py:108: UserWarning: Using generate_split_files will redirect the call to generate_ann_file. Consider directly calling that function
  warnings.warn("Using generate_split_files will redirect the call to generate_ann_file. Consider directly calling that function")


Method for CAM generation: gradcam++, eigen-smooth:False, aug-smooth:False, vit-like:True
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 8041/8041, 14.7 task/s, elapsed: 546s, ETA:     0s
Save Split file for Cams
Creating annotation file at ../preGenData/StanfordCars_Original/swinBase/GPU\valAllCAMsGrad++GPU.txt
Save generated CAMs to ../preGenData/StanfordCars_Original/swinBase/GPU\valAllCAMsGrad++GPU.npz
Method for CAM generation: xgradcam, eigen-smooth:False, aug-smooth:False, vit-like:True
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 8041/8041, 14.7 task/s, elapsed: 545s, ETA:     0s
Save Split file for Cams
Creating annotation file at ../preGenData/StanfordCars_Original/swinBase/GPU\valAllCAMsXGradGPU.txt
Save generated CAMs to ../preGenData/StanfordCars_Original/swinBase/GPU\valAllCAMsXGradGPU.npz
Method for CAM generation: ei

In [5]:
generate_cams.main([imgRootValOriginal, configCAMResOG, checkpointCAMResOG, '--device', 'cuda',
                    '--segData', segDataPathNoScaleOG,
                   '--saveBlurred', '--blurredSegments', 'background', 'hood', 'front_bumper',
                    '--segConfig', configSeg,
                    '--segCheckpoint', checkpointSeg, '-s', 'blurredBackground_Hood_FBumper', '--blurWhite'])

generate_cams.main([imgRootValOriginal, configCAMResOG, checkpointCAMResOG, '--device', 'cuda',
                    '--segData', segDataPathNoScaleOG,
                    '--saveBlurred', '--blurredSegments', 'front_left_light', 'hood', 'front_bumper',
                    '--segConfig', configSeg,
                    '--segCheckpoint', checkpointSeg, '-s', 'blurredFLLight_Hood_FBumper', '--blurWhite'])

generate_cams.main([imgRootValOriginal, configCAMResOG, checkpointCAMResOG, '--device', 'cuda',
                    '--segData', segDataPathNoScaleOG,
                    '--saveBlurred', '--blurredSegments', 'front_left_light',
                    '--segConfig', configSeg,
                    '--segCheckpoint', checkpointSeg, '-s', 'blurredFrontLeftLight', '--blurWhite'])

Generating CAMs using blurred images.
Modifying pipeline cfg so that specified segments background,hood,front_bumper will be blurred out.
Method for CAM generation: GradCAM, eigen-smooth:False, aug-smooth:False, vit-like:False
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 4.3 task/s, elapsed: 3124s, ETA:     0s
Save Split file for Cams
Creating annotation file at blurredBackground_Hood_FBumper\resultsCAM.txt
Save generated CAMs to blurredBackground_Hood_FBumper\resultsCAM.npz


C:\Users\Linus\Desktop\XAI-Vehicle-Classification\scripts\utils\io.py:108: UserWarning: Using generate_split_files will redirect the call to generate_ann_file. Consider directly calling that function
  warnings.warn("Using generate_split_files will redirect the call to generate_ann_file. Consider directly calling that function")


Generating CAMs using blurred images.
Modifying pipeline cfg so that specified segments front_left_light,hood,front_bumper will be blurred out.
Method for CAM generation: GradCAM, eigen-smooth:False, aug-smooth:False, vit-like:False
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 4.6 task/s, elapsed: 2926s, ETA:     0s
Save Split file for Cams
Creating annotation file at blurredFLLight_Hood_FBumper\resultsCAM.txt
Save generated CAMs to blurredFLLight_Hood_FBumper\resultsCAM.npz
Generating CAMs using blurred images.
Modifying pipeline cfg so that specified segments front_left_light will be blurred out.
Method for CAM generation: GradCAM, eigen-smooth:False, aug-smooth:False, vit-like:False
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>                          ] 879/13333, 4.8 task/s, elapsed: 182s, ETA:  2575s

KeyboardInterrupt: 

In [14]:
generate_blurred(configCAMResWeb, checkpointCAMResWeb, 'background', segDataPathNoScaleWeb, imgRootValWeb, annfileValWeb,
                configSeg, checkpointSeg, singleColor=[255,255,255],
                saveDir='WebblurredBackgroundWhite')
generate_blurred(configCAMResWeb, checkpointCAMResWeb, ['background', 'hood', 'front_bumper'], 
                 segDataPathNoScaleWeb, imgRootValWeb, annfileValWeb,
                 configSeg, checkpointSeg, singleColor=[255,255,255],
                 saveDir='WebblurredBackground_Hood_FBumperWhite')
generate_blurred(configCAMResWeb, checkpointCAMResWeb, ['front_left_light', 'hood', 'front_bumper'], 
                 segDataPathNoScaleWeb, imgRootValWeb, annfileValWeb,
                 configSeg, checkpointSeg, singleColor=[255,255,255],
                 saveDir='WebblurredFLLight_Hood_FBumperWhite')
generate_blurred(configCAMResWeb, checkpointCAMResWeb, ['front_left_light'], 
                 segDataPathNoScaleWeb, imgRootValWeb, annfileValWeb,
                 configSeg, checkpointSeg, singleColor=[255,255,255],
                 saveDir='WebblurredFrontLeftLightWhite')

generate_blurred(configCAMResStanford, checkpointCAMResStanford, 'background',
                 segDataPathNoScaleStanfordOG, imgRootValStandford, annfileValStanford,
                 configSeg, checkpointSeg, singleColor=[255,255,255],
                 saveDir='StanfordblurredBackgroundWhite')
generate_blurred(configCAMResStanford, checkpointCAMResStanford, ['background', 'hood', 'front_bumper'], 
                 segDataPathNoScaleStanfordOG, imgRootValStandford, annfileValStanford,
                 configSeg, checkpointSeg, singleColor=[255,255,255],
                 saveDir='StanfordblurredBackground_Hood_FBumperWhite')
generate_blurred(configCAMResStanford, checkpointCAMResStanford, ['front_left_light', 'hood', 'front_bumper'], 
                 segDataPathNoScaleStanfordOG, imgRootValStandford, annfileValStanford,
                 configSeg, checkpointSeg, singleColor=[255,255,255],
                 saveDir='StanfordblurredFLLight_Hood_FBumperWhite')
generate_blurred(configCAMResStanford, checkpointCAMResStanford, ['front_left_light'], 
                 segDataPathNoScaleStanfordOG, imgRootValStandford, annfileValStanford,
                 configSeg, checkpointSeg, singleColor=[255,255,255],
                 saveDir='StanfordblurredFrontLeftLightWhite')

Creating blurred Images in WebblurredBackground_Hood_FBumperWhite
Blurring segments into single color
Blurred Images will be saved to WebblurredBackground_Hood_FBumperWhite
[>>>>>>>>>>>>>>>>>>>>>>>>] 15627/15627, 34.0 task/s, elapsed: 460s, ETA:     0sCreating blurred Images in WebblurredFLLight_Hood_FBumperWhite
Blurring segments into single color
Blurred Images will be saved to WebblurredFLLight_Hood_FBumperWhite
[>>>>>>>>>>>>>>>>>>>>>>>>] 15627/15627, 34.6 task/s, elapsed: 452s, ETA:     0sCreating blurred Images in WebblurredFrontLeftLightWhite
Blurring segments into single color
Blurred Images will be saved to WebblurredFrontLeftLightWhite
[>>>>>>>>>>>>>>>>>>>>>>>>] 15627/15627, 34.5 task/s, elapsed: 453s, ETA:     0sCreating blurred Images in StanfordblurredBackground_Hood_FBumperWhite
Blurring segments into single color
Blurred Images will be saved to StanfordblurredBackground_Hood_FBumperWhite
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 8041/8041, 47.2 task/s, elapsed: 170s, ETA:     0sCreati

In [ ]:
"""CompCars_Original"""
igR = blurWhiteBackgroundImgRootOriginal
cfgC = configCAMResOG
ckpC = checkpointCAMResOG
svPre = '../preGenData/blurredBackgroundWhite/CompCars_Original/ResNet/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++', '--method', 'gradcam++', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigen', '--method', 'eigencam', '--device', 'cuda'])


cfgC = configCAMSwinBaseOG
ckpC = checkpointCAMSwinBaseOG
svPre = '../preGenData/blurredBackgroundWhite/CompCars_Original/SwinBase/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++', '--method', 'gradcam++', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigen', '--method', 'eigencam', '--vit-like', '--device', 'cuda'])


cfgC = configCAMSwinSmallOG
ckpC = checkpointCAMSwinSmallOG
svPre = '../preGenData/blurredBackgroundWhite/CompCars_Original/SwinSmall/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++', '--method', 'gradcam++', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigen', '--method', 'eigencam', '--vit-like', '--device', 'cuda'])


"""CompCars_Web"""
igR = blurWhiteBackgroundImgRootWeb
cfgC = configCAMResWeb
ckpC = checkpointCAMResWeb
svPre = '../preGenData/blurredBackgroundWhite/CompCars_Web/ResNet/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++', '--method', 'gradcam++', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigen', '--method', 'eigencam', '--device', 'cuda'])


cfgC = configCAMSwinBaseWeb
ckpC = checkpointCAMSwinBaseWeb
svPre = '../preGenData/blurredBackgroundWhite/CompCars_Web/SwinBase/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++', '--method', 'gradcam++', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigen', '--method', 'eigencam', '--vit-like', '--device', 'cuda'])


cfgC = configCAMSwinSmallWeb
ckpC = checkpointCAMSwinSmallWeb
svPre = '../preGenData/blurredBackgroundWhite/CompCars_Web/SwinSmall/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++', '--method', 'gradcam++', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigen', '--method', 'eigencam', '--vit-like', '--device', 'cuda'])


"""StanfordCars_Original"""
igR = blurWhiteBackgroundImgRootStanford
cfgC = configCAMResStanford
ckpC = checkpointCAMResStanford
svPre = '../preGenData/blurredBackgroundWhite/StanfordCars_Original/ResNet/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++', '--method', 'gradcam++', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigen', '--method', 'eigencam', '--device', 'cuda'])


cfgC = configCAMSwinBaseStanford
ckpC = checkpointCAMSwinBaseStanford
svPre = '../preGenData/blurredBackgroundWhite/StanfordCars_Original/SwinBase/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++', '--method', 'gradcam++', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigen', '--method', 'eigencam', '--vit-like', '--device', 'cuda'])


cfgC = configCAMSwinSmallStanford
ckpC = checkpointCAMSwinSmallStanford
svPre = '../preGenData/blurredBackgroundWhite/StanfordCars_Original/SwinSmall/'
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsGrad++', '--method', 'gradcam++', '--vit-like', '--device', 'cuda'])
generate_cams.main([igR, cfgC, ckpC, '-s', svPre + 'valAllCAMsEigen', '--method', 'eigencam', '--vit-like', '--device', 'cuda'])

Method for CAM generation: gradcam++, eigen-smooth:False, aug-smooth:False, vit-like:False
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 24.1 task/s, elapsed: 554s, ETA:     0s
Save Split file for Cams
Creating annotation file at ../preGenData/blurredBackgroundWhite/CompCars_Original/ResNet\valAllCAMsGrad++.txt
Save generated CAMs to ../preGenData/blurredBackgroundWhite/CompCars_Original/ResNet\valAllCAMsGrad++.npz


C:\Users\Linus\Desktop\XAI-Vehicle-Classification\scripts\utils\io.py:108: UserWarning: Using generate_split_files will redirect the call to generate_ann_file. Consider directly calling that function
  warnings.warn("Using generate_split_files will redirect the call to generate_ann_file. Consider directly calling that function")


Method for CAM generation: eigencam, eigen-smooth:False, aug-smooth:False, vit-like:False
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 3.4 task/s, elapsed: 3922s, ETA:     0s
Save Split file for Cams
Creating annotation file at ../preGenData/blurredBackgroundWhite/CompCars_Original/ResNet\valAllCAMsEigen.txt
Save generated CAMs to ../preGenData/blurredBackgroundWhite/CompCars_Original/ResNet\valAllCAMsEigen.npz
Method for CAM generation: gradcam++, eigen-smooth:False, aug-smooth:False, vit-like:True
Generate Results for specified files
Automatically choose the last norm layer as target_layer.
[>>>>>>>>>>>>>>>>>>>>>>>] 13333/13333, 12.5 task/s, elapsed: 1064s, ETA:     0s
Save Split file for Cams
Creating annotation file at ../preGenData/blurredBackgroundWhite/CompCars_Original/SwinBase\valAllCAMsGrad++.txt
Save generated CAMs to ../preGenData/blurredBackgroundWhite/CompCars_Original/SwinBase\valAl